In [4]:
import torch
import h5py
import sys
sys.path.insert(0, '../')
from utils.gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch("0")

# Define new model
# Load augmentation outputs for MNIST 
train_path = '../birds200/five_crop_hflip_scale/model_outputs/val/resnet18_val.h5'
test_path = '../birds200/five_crop_hflip_scale/model_outputs/val/resnet18_test.h5'

train = h5py.File(train_path, 'r')
test = h5py.File(test_path, 'r')
dataset = 'birds200'
agg_name = 'full_lr'
aug_name = 'combo'
model_name ='resnet18'
n_classes = 200
n_augs = 30
orig_idx = 12

Using GPU:0


In [2]:
from tta_agg_models import TTARegression
agg_models_dir = '../' + dataset + '/five_crop_hflip_scale/agg_models/'
model_path = agg_models_dir +model_name+'/'+aug_name + '/full_lr.pth'
model = TTARegression(30,n_classes,1, 'even')
model.load_state_dict(torch.load(model_path))
model.eval()


TTARegression()

In [6]:
train_preds = train['batch1_inputs'].values
val_preds = test['batch1_inouts'].values

(30, 2897, 200)

In [12]:
# Calculate % of the time expert is correct

((30, 2897, 200), (30, 2897, 200))

In [4]:
train['batch1_inputs']

<KeysViewHDF5 ['batch1_inputs', 'batch1_labels']>

In [46]:
# process image; output softmax prob; weight outputs based on the two

True

In [ ]:
# Define class that does aggregation + deferral prediction
class DeferredRegression(nn.Module):
    def __init__(self, n_augs, n_classes, orig_idx, temp_scale=1, initialization='even'):
        super().__init__()
        
        self.coeffs = nn.Parameter(torch.randn((n_augs, n_classes), requires_grad=True, dtype=torch.float))
        self.mix_coeffs = nn.Parameter(torch.randn(2), requires_grad=True, dtype=torch.float)
        self.temperature = temp_scale
        self.orig_idx = orig_idx
        if initialization == 'even':
            self.coeffs.data.fill_(1.0/n_augs) 
        elif initialization== 'original':
            self.coeffs.data[0,:].fill_(1)
            self.coeffs.data[1,:].fill_(0)
    
    def forward(self, x):
        # Computes the outputs / predictions
        x = x/self.temperature
        mult = self.coeffs * x
        aug_pred = mult.sum(axis=1)
        orig_pred = x[self.orig_idx]
        final_pred = torch.cat([orig_pred, aug_pred], axis=0)*self.mix_coeffs
        final_pred = final_pred.sum(axis=1)
        # Function to map augmentation outputs --> deferral value
        # Function to map augmentation outputs --> aggregated output
        # Normalize to output of [aggregated output, deferral prob]
        # Loss function based on this deferral prob and original model predictions
        return final_pred

In [ ]:
# Train this model on the validation set
epochs = 100

In [ ]:
# TEst on the corresponding test files, see if the deferrals make sense 